In [ ]:
!git clone https://github.com/qwopqwop200/GPTQ-for-LLaMa.git

In [ ]:
#downloading all the 7B file using https://github.com/shawwn/llama-dl/blob/main/llama.sh

%%bash

PRESIGNED_URL="https://agi.gpt4.org/llama/LLaMA/*"

MODEL_SIZE="7B"  # edit this list with the model sizes you wish to download
TARGET_FOLDER="./"             # where all files should end up

declare -A N_SHARD_DICT

N_SHARD_DICT["7B"]="0"
N_SHARD_DICT["13B"]="1"
N_SHARD_DICT["30B"]="3"
N_SHARD_DICT["65B"]="7"

echo "Downloading tokenizer"
wget ${PRESIGNED_URL/'*'/"tokenizer.model"} -O ${TARGET_FOLDER}"/tokenizer.model"
wget ${PRESIGNED_URL/'*'/"tokenizer_checklist.chk"} -O ${TARGET_FOLDER}"/tokenizer_checklist.chk"

(cd ${TARGET_FOLDER} && md5sum -c tokenizer_checklist.chk)

for i in ${MODEL_SIZE//,/ }
do
    echo "Downloading ${i}"
    mkdir -p ${TARGET_FOLDER}"/${i}"
    for s in $(seq -f "0%g" 0 ${N_SHARD_DICT[$i]})
    do
        wget ${PRESIGNED_URL/'*'/"${i}/consolidated.${s}.pth"} -O ${TARGET_FOLDER}"/${i}/consolidated.${s}.pth"
    done
    wget ${PRESIGNED_URL/'*'/"${i}/params.json"} -O ${TARGET_FOLDER}"/${i}/params.json"
    wget ${PRESIGNED_URL/'*'/"${i}/checklist.chk"} -O ${TARGET_FOLDER}"/${i}/checklist.chk"
    echo "Checking checksums"
    (cd ${TARGET_FOLDER}"/${i}" && md5sum -c checklist.chk)
done

In [ ]:
!mv 7B GPTQ-for-LLaMa/7B
%cd GPTQ-for-LLaMa

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
import torch
print(torch.__version__)

In [ ]:
!pip install -r requirements.txt
!python3 setup_cuda.py install

In [ ]:
!pip install accelerate

In [ ]:
!python convert_llama_weights_to_hf.py --input_dir . --model_size 7B --output_dir ./output

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python llama.py ./output/llama-7b ptb --wbits 4 --groupsize 128 --save_safetensors llama7b-4bit-128g.safetensors

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python llama_inference.py ./output/llama-7b --wbits 4 --groupsize 128 --load llama7b-4bit-128g.pt --text "this is llama"